# Introduction
In this Notebook the transferfee data for the Season 2019/2020 will be scraped from [transfermarkt.ch](https://www.transfermarkt.ch/transfers/saisontransfers/statistik/top/saison_id/2019/transferfenster/alle/land_id//ausrichtung//spielerposition_id//altersklasse//leihe//plus/1/galerie/0/page/1). This will be the dependent variable in the analysis later. The webscraping will be done with the `requests` and `BeautifulSoup` libraries.

In [ ]:
#import packages
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

# Webscraping
Since each page only shows the data for 25 players, we will need to iterate through all pages. For this, we first will need to get the last page on which there is transfer data.

In [ ]:
#This is used so transfermarkt thinks we are a webbrowser and not a scrapper
headers = {'User-Agent': 
           'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}
#webpage
page = "https://www.transfermarkt.ch/transfers/saisontransfers/statistik/top/saison_id/2019/transferfenster/alle/land_id//ausrichtung//spielerposition_id//altersklasse//leihe//plus/1/galerie/0/page/1"
#grabs the code of the page
pageTree = requests.get(page, headers=headers)
#parses the website code into html
pageSoup = BeautifulSoup(pageTree.content, 'html.parser')


In [ ]:
#Find text which indicates last page on which there is data
Basics = pageSoup.find_all("li", {"class": "letzte-seite"})

In [ ]:
#Get part of html code where last page is indicated
a=Basics[0]['title']

In [ ]:
#We only need the number, so we can delete all other text
res = a.partition("(Seite")[2]

In [ ]:
#With the regex function we can now find just the digits in the text
import re
last_Site=int(re.search(r'\d+', res).group())

In [ ]:
#get a list with all page numbers to iterate through later
all_pages=list(range(1,last_Site+1))

In [ ]:
#Create empty lists to store webscraping data for dataframe later
Basics=[]
age=[]
club=[]

In [ ]:
#iterate through every page and get the relevant data
for i in all_pages:

    #webpage
    page = "https://www.transfermarkt.ch/transfers/saisontransfers/statistik/top/saison_id/2019/transferfenster/alle/land_id//ausrichtung//spielerposition_id//altersklasse//leihe//plus/1/galerie/0/page/"+str(i)
    #grabs the code of the page
    pageTree = requests.get(page, headers=headers)
    #parses the website code into html
    pageSoup = BeautifulSoup(pageTree.content, 'html.parser')
    
    Basics.append(pageSoup.find_all("td", {"class": "hauptlink"}))#get transfer fee
    age.append(pageSoup.find_all("td", {"class":"zentriert"}))#get age of player
    club.append(pageSoup.find_all("a", {"class": "vereinprofil_tooltip"}))#get full club names

# Get data from HTML code
Now that we have the html code, we can get the data we need for the Analysis from it.

In [ ]:
#Let's look at the first Values
Basics[0][0].text
age[0][1].text
club[0][1].text


In [ ]:
#Since every 4th item on web page is a new player, we create a list with every 4th item
lst=list(range(len(Basics)))
lst[0::4]

In [ ]:
#Create function that cleanes the name list
def name_cleaner(lst):
    aux_list=[] #Create auxillary list
    for i in lst: #Go through Names list and replace the "\n" in the beginning and empty space in end
        i=i[:-1]
        aux_list.append(i.replace("\n",""))
    return aux_list

In [ ]:
#Create List of Names
Names=[]
for page in range(len(Basics)):
    lst=list(range(len(Basics[page])))
    for i in lst[0::4]:
        Names.append(Basics[page][i].text) #This is done to replace all the uncessary stuff from the Leauge Name
Names=name_cleaner(Names)    

In [ ]:
#Create List of Selling clubs
Left=[]

for page in range(len(club)):#iterate through all pages
    lst=list(range(len(club[page])))#iterate through whole page
    for i in lst[0::4]:#Every 4th item new selling club
        Left.append(club[page][i].find("img")["alt"]) #Get selling club
Left                 

In [ ]:
#Create List of Buying clubs
Joined=[]

for page in range(len(club)):
    lst=list(range(len(club[page])))
    for i in lst[2::4]:
        Joined.append(club[page][i].find("img")["alt"]) #Get buying club
Joined                 

In [ ]:
#Create List of transfer fee
Fee=[]

for page in range(len(Basics)):
    lst=list(range(len(Basics[page])))
    for i in lst[3::4]:
        Fee.append(Basics[page][i].text) #Get transfer fee
Fee

In [ ]:
#Create list with ages
age_list=[]

for page in range(len(age)):
    lst=list(range(len(age[page])))
    for i in lst[1::3]:
        age_list.append(int(age[page][i].text)) #get age

In [ ]:
#Get player ID on transfermarkt.ch
ID=[]

for page in range(len(Basics)):
    lst=list(range(len(Basics[page])))
    for i in lst[0::4]:
        ID.append(Basics[page][i].find("a")["id"]) #get player ID
ID

# Data Handling
Now that we have all the data we need in columns, we can create the transfer fee dataframe and perform the necessary data transformations.

In [ ]:
#Create Data Frame from the lists
df = pd.DataFrame({"Player":Names,"ID":ID,"Age":age_list,"Left":Left,"Joined":Joined,"Fee":Fee})
df

In [ ]:
#Create list with True for all players that were loaned or left the club for free, since these data points are irrelevant for our analysis
Loans=[]#empty list
for i in df.Fee:#iterate through all transfer fees
    Loans.append("e" in i) #since all instances we don't need have a "e" in them (and only those instances), we get those items

In [ ]:
#get indices and drop players
indexNames = df[Loans].index
df.drop(indexNames , inplace=True)

In [ ]:
#Since transfer fees are strings, we need to transform them to integer type strings into int and create new list with integers
New_List=[]
for i in df.Fee:
    if "Mrd" in i:
        i=i.replace(' Mrd. €', '')
        i=i.replace(',', '.')
        New_List.append(int(float(i)*1000000000))
    elif "Mio" in i:
        i=i.replace(' Mio. €', '')
        i=i.replace(',', '.')
        New_List.append(int(float(i)*1000000))
    elif "Tsd" in i:
        i=i.replace(' Tsd. €', '')
        i=i.replace(',', '.')
        New_List.append(int(float(i)*1000))
    else:
        New_List.append(np.nan)

In [ ]:
#drop old transfer fees that are strings
df.drop("Fee", axis = 1, inplace = True)

# Add inteeger transfer fees
df["Fee"] = New_List

In [ ]:
#drop all np.nan
df.dropna(inplace=True)

In [ ]:
#egt all duplicate entries
a=df[df.ID.duplicated()]

In [ ]:
#Get list with all inicices of duplicate entries
duplicates=[]
for i in range(a.shape[0]):
    duplicates.append(df.where(df["ID"]==a.iloc[i]["ID"]).dropna().index)
duplicates

In [ ]:
#go through all plyers that are in dataframe twice (cause they were bought and resold in same transfer window)
for i in duplicates:
    df.drop(df.loc[i]["Fee"].idxmin(), axis=0,inplace=True)#drop lower transferfee

In [ ]:
#drop all duplicates in df
df.drop_duplicates(inplace=True)

In [ ]:
#set player name as index
df.index=df["Player"]
df.drop("Player", axis=1, inplace=True)

In [ ]:
df

In [ ]:
#save df to pickle
df.to_pickle("Transfers.pkl")